In [1]:
import pandas as pd
path = "/Users/Desktop/data/lastfm-2k/"

artists_df = pd.read_csv(path+'artists.dat', sep='\t', header=0, names=['artistID', 'name', 'url', 'pictureURL'])

tags_df = pd.read_csv(path+'tags.dat', sep='\t', header=0, encoding='ISO-8859-1')
# print("\nTags DataFrame:")


user_artists_df = pd.read_csv(path+'user_artists.dat', sep='\t', header=0)
# print("\nUser Artists DataFrame:")
# print(user_artists_df.head())

user_taggedartists_df = pd.read_csv(path+'user_taggedartists.dat', sep='\t', header=0)
# print("\nUser Tagged Artists DataFrame:")
# print(user_taggedartists_df.head())

user_taggedartists_timestamps_df = pd.read_csv(path+'user_taggedartists-timestamps.dat', sep='\t', header=0)
# print("\nUser Tagged Artists Timestamps DataFrame:")
# print(user_taggedartists_timestamps_df.head())

user_friends_df = pd.read_csv(path+'user_friends.dat', sep='\t', header=0)
# print("\nUser Friends DataFrame:")
# print(user_friends_df.head())

artist_tags = user_taggedartists_timestamps_df.groupby('artistID')['tagID'].apply(lambda x: list(set(x))).reset_index()
artist_tags['tagCount'] = artist_tags['tagID'].apply(len)
artists = pd.merge(artists_df, artist_tags, how='right', on="artistID")  

def get_tag_text(tag_ids):
    tag_texts = [tags_dict[tag_id] for tag_id in tag_ids if tag_id in tags_dict]
    return tag_texts

tags_dict = pd.Series(tags_df.tagValue.values, index=tags_df.tagID).to_dict()
artists['tagID_text'] = artists['tagID'].apply(get_tag_text)

In [14]:
artists

,artistID,name,url,pictureURL,tagID,tagCount,tagID_text,item_id
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg,"[233, 190, 14, 18, 179, 727, 10137, 127, 61, 3...",11,"[dark, gothic rock, ambient, electronic, gothi...",2001
5,6,Moonspell,http://www.last.fm/music/Moonspell,http://userserve-ak.last.fm/serve/252/2181591.jpg,"[191, 4, 7, 9640, 233, 1482, 713, 524, 1484, 8...",16,"[instrumental, black metal, gothic metal, baza...",503
6,7,Marilyn Manson,http://www.last.fm/music/Marilyn+Manson,http://userserve-ak.last.fm/serve/252/2558217.jpg,"[1, 2, 386, 10116, 6, 7, 524, 793, 797, 285, 5...",73,"[metal, alternative metal, heavy metal, great ...",910
7,8,DIR EN GREY,http://www.last.fm/music/DIR+EN+GREY,http://userserve-ak.last.fm/serve/252/46968835...,"[1, 1219, 552, 73, 138, 139, 2155, 141, 233, 3...",11,"[metal, jrock, weeabo, rock, hard rock - metal...",179
8,9,Combichrist,http://www.last.fm/music/Combichrist,http://userserve-ak.last.fm/serve/252/51273485...,"[9, 12, 18, 1447, 562, 563, 565, 309, 570, 571...",22,"[electro-industrial, aggrotech, electronic, re...",1534
...,...,...,...,...,...,...,...,...
9565,13292,Don Omar,http://www.last.fm/music/Don+Omar,http://userserve-ak.last.fm/serve/252/27847067...,"[8301, 304, 370, 723, 8310]",5,"[lpa, hip hop, favorite, reggaeton, viaje de e...",2789
9569,13303,Tego Calderon,http://www.last.fm/music/Tego+Calderon,http://userserve-ak.last.fm/serve/252/4069912.jpg,[723],1,[reggaeton],2790
9903,13888,Tindersticks,http://www.last.fm/music/Tindersticks,http://userserve-ak.last.fm/serve/252/262479.jpg,"[195, 6628, 6629, 134, 78, 79, 1198, 81, 24]",9,"[british, barkbarkdisco, bark, singer-songwrit...",2797
10313,14645,SoKo,http://www.last.fm/music/SoKo,http://userserve-ak.last.fm/serve/252/62236511...,"[321, 130, 11489, 228, 292, 134, 296, 81, 1753...",13,"[french, female vocalists, my own private suns...",2805


In [2]:
import numpy as np

item_list = path + 'item_map.txt'

item_map = {}

with open(item_list, 'r') as file:
    for line in file:
        key, value = line.strip().split()
        item_map[int(key)] = int(value)


artists = artists[artists['artistID'].isin(set(item_map.keys()))]
artists['item_id'] = artists['artistID'].apply(lambda x: item_map[x])
# artists = artists[['tagID_text', 'artistID', 'item_id']]

In [8]:

import copy

item_1stneighbor_text_dic = {}
triples = [] 

single_cols = []
multiple_cols = ['tagID_text']

for index, row in artists.iterrows():
    triples0 = [] 
    item_id = row['item_id']
    item_name = row['name']

    for col in multiple_cols:
        tags = row[col]
        # triples0.append((item_name, col, row[col]))
        for entity in row[col]:
            triples.append((item_name, col, entity))  
            triples.append((entity, col+"_reverse", item_name)) 

    # if len(single_cols) > 0:
    #     for col in single_cols:
    #         if col not in ['city', 'state','address', 'stars']:
    #             relation = col.split(".")[-1]
    #         else:
    #             relation = col
                
    #         value = row[col]
    #         if (isinstance(value, (float, int)) and not np.isnan(value)) or pd.notna(value): 
    #             triples0.append((business_name, relation, row[col]))
    #             if relation in ["city", "categories", ]:
    #                 triples.append((business_id, relation, row[col]))
    #                 triples.append((row[col], relation+"_", business_id))
    #              # 人工组合column
    #             if relation in ['stars', "GoodForKids", "RestaurantsDelivery", "RestaurantsTakeOut",
    #                                     "Alcohol"]:
                  
    #                 relation = "city_"+relation
    #                 triples.append((business_id, relation, row['city']+"_"+str(row[col])))
    #                 triples.append((row['city']+"_"+str(row[col]), relation+"_", business_id))
    
        
    item_1stneighbor_text_dic[item_id] = f'The artist {item_name} has been tagged by users with the following labels: {tags}'
    

data = pd.DataFrame(triples, columns=['head', 'relation', 'tail'])
data = data.dropna()

In [10]:
# print(item_1stneighbor_text_dic[0])

The artist Duran Duran has been tagged by users with the following labels: ['romantic', 'u2', 'eighties', 'johnny', 'dd', 'chillout', 'ambient', 'new wave', 'synth pop', 'electronic', "80's", 'erotic', 'favourite songs', '1995', 'pop', '80s', '1981', 'atmospheric', 'ethereal', 'experimental', 'danny', 'dance', 'synth-pop', 'melancholic', 'duran my fave from wedding album', 'love duran jt', 'acoustic rock', 'dub', 'rock', 'synthpop', 'covers', 'rain', 'alternative rock', 'alternative', 'indie', 'indie rock', 'house', 'pop rock', 'nelly furtado', 'slowies', 'edit', 'uk', 'old school', 'song with samples', 'seen live', 'hit', 'guilty pleasure', 'singer-songwriter', 'best', 'numbers', 'faves', 'scifi', 'perfect', 'all time faves', 'blue songs', 'grunge', 'madonna', 'bond', 'soundtrack', 'acoustic', 'electronica', 'instrumental', 'classic rock', '2008', 'british', 'anthemic', 'stylish', 'club', 'hot', 'guilty pleasures', 'majestic', 'james bond', '80s stuff', 'legend', '00s', 'beautiful', '

In [11]:

import dgl
import random
import torch
from collections import Counter
from sklearn.preprocessing import LabelEncoder


def create_kg_from_data(data):
  
    node_encoder = LabelEncoder()
    relation_encoder = LabelEncoder()
    
    nodes = pd.concat([data['head'], data['tail']]).unique()
    node_encoder.fit(nodes)
    head = node_encoder.transform(data['head'])
    tail = node_encoder.transform(data['tail'])

    relations = data['relation'].unique()
    relation_encoder.fit(relations)
    relation = relation_encoder.transform(data['relation'])

    g = dgl.heterograph({
        ('node', 'relation', 'node'): (head, tail)
    })
    
    g.edges['relation'].data['etype'] = torch.tensor(list(relation))  
    return g, node_encoder, relation_encoder



def node2vec_walk(g, start_node, K):
    walk_2nd_neighbor_dic = {}
    walk = [start_node]
    visited_neighbors = set()
    visited_second_neighbors = set()
    
    one_hop_neighbors = g.successors(start_node).tolist()
    
    for neighbor in one_hop_neighbors:
        walk_2nd_neighbor_dic[neighbor] = []
        walk.append(neighbor)
        visited_neighbors.add(neighbor)
        
        second_order_neighbors = g.successors(neighbor).tolist()
        second_order_neighbors = [n for n in second_order_neighbors if n != start_node]
        # print("debug second_order_neighbors", second_order_neighbors)
        if len(second_order_neighbors) > K:
            sampled_neighbors = np.random.choice(second_order_neighbors, K, replace=False)
        else:
            sampled_neighbors = second_order_neighbors
        
        for second_neighbor in sampled_neighbors:
            walk.append(second_neighbor)
            visited_second_neighbors.add(second_neighbor)
            walk_2nd_neighbor_dic[neighbor].append(second_neighbor)
    
    return walk_2nd_neighbor_dic

def get_edge_types(g, walk):
    edge_types = []
    for i in range(len(walk) - 1):
        src, dst = walk[i], walk[i + 1]
        eid = g.edge_ids(src, dst)
        edge_type = g.edata['etype'][eid]
        edge_types.append(edge_type.item())
    return edge_types


# -------------------------------------main----------------------------------
data["tail"] = data["tail"].apply(str)
data["head"] = data["head"].apply(str)
graph, node_encoder, relation_encoder = create_kg_from_data(data)

item_id_name_map = artists.set_index('item_id')['name'].to_dict()
item_2ndneighbor_text_dic = {}
cnt = 0

selected_relation = [ "tagID_text",]

# for item_id in list(item_id_name_map.keys())[:10]:
for item_id in item_id_name_map.keys():
    cnt += 1
    if cnt % 100 == 0:
        print(cnt)
        # break
    
    total_text = []

    item_name = item_id_name_map[item_id]
    item_node_id = node_encoder.transform([str(item_name)])[0]
    item_2ndneighbor_text_dic[item_id] = []


    cur_dic = node2vec_walk(graph, item_node_id, K=5)
    
    for neighbor_id in cur_dic.keys():
        
        relation = get_edge_types(graph, walk=[item_node_id, neighbor_id])
        # to text
        neighbor = node_encoder.inverse_transform([neighbor_id])[0]
        relation = relation_encoder.inverse_transform(relation)[0]
        
        if relation not in selected_relation:
            continue
     
        second_order_neighbors = cur_dic[neighbor_id]
        second_order_neighbors_list = []
        for second_order_neighbor_id in second_order_neighbors:
            second_order_neighbor = node_encoder.inverse_transform([second_order_neighbor_id])[0]
            second_order_neighbors_list.append(second_order_neighbor)
        # second_order_neighbors_name_list = [item_id_name_map[int(i)]+"@"+business[business["business_id_mapped"]==int(i)]['address'].values[0] for i in second_order_neighbors_list]
        second_order_neighbors_name_list = [i for i in second_order_neighbors_list]
        item_2ndneighbor = "; ".join(second_order_neighbors_name_list)

        city = neighbor.split('_')[0]
        if relation == "tagID_text":
            neighbor = neighbor
            text = f"Artists with the same tag ({neighbor}) also include: {item_2ndneighbor}."
        else:
            continue
            # raise ValueError("Unknown relation type") 

        total_text.append(text)
    
    item_2ndneighbor_text_dic[item_id] = " ".join(total_text) 

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [5]:
item_2ndneighbor_text_dic[0]

"Artists with the same tag (romantic) also include: blink-182; Charles Aznavour; Cyndi Lauper; BoA; Moenia; Matchbox Twenty; Bruno Mars; Nickelback; Hubert Kah; Ne-Yo. Artists with the same tag (u2) also include: PJ Harvey; Gang of Four; Garbage; Robbie Williams; The Beatles; Zero 7; The Smiths; The Bravery; Eels; Rilo Kiley. Artists with the same tag (eighties) also include: David Bowie; Thompson Twins; Kylie Minogue; Bauhaus; Hubert Kah; ABBA; Anything Box; Kraftwerk; A Flock of Seagulls; Wham!. Artists with the same tag (johnny) also include: Sade; David Bowie; Queensrÿche; Oingo Boingo. Artists with the same tag (dd) also include: . Artists with the same tag (chillout) also include: Pet Shop Boys; Elbow; Yann Tiersen; Massive Attack; Leonard Cohen; HIM; Solar Fields; Orbital; Quantic; Autechre. Artists with the same tag (ambient) also include: Devin Townsend; This Will Destroy You; Aphex Twin; Oi Va Voi; Deep Forest; Efterklang; God Is an Astronaut; Sigur Rós; Emancipator; The Chem

In [19]:
import json

def prompt_generation(id, name, item_1stneighbor_text_dic, item_2ndneighbor_text_dic):
    basic_info = item_1stneighbor_text_dic[id]
    order_2nd_info = item_2ndneighbor_text_dic[id]


    analysis = \
    f"""
    The name of the artist is: {name}. Below is the information about this artist:
    1) Tags of this artist: {basic_info} ;
    2) Related artists who have been tagged same labels with this artist: {order_2nd_info}
    
    """
    
    return analysis


question_dic = {}
for id, name in item_id_name_map.items():
    prompt = prompt_generation(id, name, item_1stneighbor_text_dic, item_2ndneighbor_text_dic)        
    question_dic[id] = prompt


print("The number of requests is: ", len(list(question_dic.keys())))



with open('/Users/Desktop/data/lastfm-2k/llm_input_item.json', 'w') as f:
    json.dump(question_dic, f)

The number of requests is:  2813
